In [16]:
#Importaciones
import cv2 as cv
import mediapipe as mp
import numpy as np
import tensorflow as tf
import os
import pandas as pd

#Inicializar utilidad de dibujo para graficar lineas
mp_drawing= mp.solutions.drawing_utils
#Inicializa el modulo de estimacion de pose (rastrea los puntos)
mp_pose=mp.solutions.pose

In [17]:
#Cargar el modelo de entrenamiento
# Verificación de la ruta del modelo
# model_path = r'D:\IAProyectos\ProyectoFinal_IA\models\mnist_model.h5'
model_path = r'D:\IAProyectos\ProyectoFinal_IA\models\mnist_model_mejorado.h5'
if not os.path.exists(model_path):
    print(f"El modelo en {model_path} no existe.")
else:
    print(f"El modelo en {model_path} existe.")

try:
    # Cargar el modelo de entrenamiento
    model = tf.keras.models.load_model(model_path)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")

# Mapear índices a nombres de clase
# label_to_index = {'Camiseta': 0,'Pantalón': 1,'Suéter': 2,'Vestido': 3,'Suéter': 4,'Zapatilla': 5,
#                     'Camiseta': 6,'Zapatilla': 7,'Camiseta': 8,'Zapatilla': 9}

label_to_index = {'Camiseta': 0,'Pantalón': 1,'Suéter': 2,'Vestido': 3,'Abrigo': 4,'Sandalia': 5,'Camisa': 6,
                    'Zapatilla': 7,'Bolso': 8,'Bota': 9}

index_to_label = {v: k for k, v in label_to_index.items()}

El modelo en D:\IAProyectos\ProyectoFinal_IA\models\mnist_model_mejorado.h5 existe.
Modelo cargado exitosamente.


<h2>Puntos de interes</h2>

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px">


<h2>Definir rangos de color HSV y establecer los puntos de interes a analizar</h2>

In [18]:
# Definir rangos de colores en HSV
"""
Primer par: (0, 50, 50) a (10, 255, 255)
    -Hue: 0 a 10 (Los tonos de rojo al principio del espectro)
    -Saturation: 50 a 255 (De un rojo ligeramente apagado a un rojo muy saturado)
    -Value: 50 a 255 (De un rojo oscuro a un rojo muy brillante)
"""
color_ranges = {
    "Rojo": [(0, 50, 50), (10, 255, 255), (170, 50, 50), (180, 255, 255)],
    "Naranja": [(11, 50, 50), (25, 255, 255)],
    "Amarillo": [(26, 50, 50), (35, 255, 255)],
    "Verde Claro": [(36, 50, 50), (70, 255, 255)],
    "Verde Oscuro": [(71, 50, 50), (85, 255, 255)],
    "Cian": [(86, 50, 50), (95, 255, 255)], #Un azul medio claro
    "Azul Claro": [(96, 50, 50), (110, 255, 255)],
    "Azul Oscuro": [(111, 50, 50), (125, 255, 255)],
    "Violeta Claro": [(126, 50, 50), (145, 255, 255)],
    "Violeta Oscuro": [(146, 50, 50), (160, 255, 255)],
    "Rosa": [(161, 50, 50), (169, 255, 255)],
    "Marrón Claro": [(10, 100, 20), (20, 255, 200)],
    "Marrón Oscuro": [(10, 100, 20), (20, 255, 150)],
    "Blanco": [(0, 0, 200), (180, 20, 255)],
    "Gris Claro": [(0, 0, 150), (180, 50, 200)],
    "Gris Oscuro": [(0, 0, 50), (180, 50, 150)],
    "Negro": [(0, 0, 0), (180, 255, 50)],
    "Beige": [(20, 20, 70), (35, 255, 255)],
    "Oro": [(20, 100, 100), (30, 255, 255)],
    "Plata": [(0, 0, 180), (180, 20, 255)],
    "Lima": [(75, 100, 100), (85, 255, 255)],
    "Turquesa": [(85, 150, 100), (95, 255, 255)], # entre cian y azul claro.
    "Lavanda": [(120, 50, 100), (140, 255, 255)], #violeta claro.
    "Coral": [(0, 50, 100), (15, 255, 255)], #entre rosa y naranja
    "Menta": [(50, 100, 100), (70, 255, 255)],
    "Durazno": [(10, 50, 100), (20, 255, 255)]
}


# Lista de puntos de interés / funciona correctamente
puntos_interes = [11, 12, 13, 14, 23, 24, 25, 26, 27, 28, 30, 32, 29, 31]

# Conexiones relevantes
conexiones_interes = [
    (11, 12), (12, 14), (11, 13), (24, 23), (24, 26), (23, 25),
    (12, 24), (11, 23), (24, 23), (26, 28), (25, 27), (32, 30), 
    (30, 32), (27, 29)
]

# Lista de puntos por zonas
tronco_superior = [11, 12, 13, 14, 23, 24]
tronco_inferior = [23, 24, 25, 26, 27, 28]
pies = [28, 32, 30, 27, 29, 31]

<h2>Funciones</h2>

In [19]:
def obtener_nombre_color(hsv_color):
    h, s, v = hsv_color
    for color, ranges in color_ranges.items():
        for i in range(0, len(ranges), 2):
            if ranges[i][0] <= h <= ranges[i+1][0] and ranges[i][1] <= s <= ranges[i+1][1] and ranges[i][2] <= v <= ranges[i+1][2]:
                return color
    return "Desconocido"

# Función para dibujar las bounding boxes
def dibujar_bounding_box(image, puntos):
    x_coords = [punto.x for punto in puntos]
    y_coords = [punto.y for punto in puntos]
    x_min = int(min(x_coords) * image.shape[1])
    x_max = int(max(x_coords) * image.shape[1])
    y_min = int(min(y_coords) * image.shape[0])
    y_max = int(max(y_coords) * image.shape[0])
    cv.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return (x_min, y_min, x_max, y_max)

def dibujar_bounding_box_resize(image, landmarks, expand_factor=0.2):
    x_coords = [int(landmark.x * image.shape[1]) for landmark in landmarks]
    y_coords = [int(landmark.y * image.shape[0]) for landmark in landmarks]
    x1, x2 = min(x_coords), max(x_coords)
    y1, y2 = min(y_coords), max(y_coords)

    # Expandir la bounding box horizontalmente
    width = x2 - x1
    expand_width = int(width * expand_factor)
    x1 = max(0, x1 - expand_width)
    x2 = min(image.shape[1], x2 + expand_width)

    cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return x1, y1, x2, y2

# Función para obtener el color dominante en HSV con suavizado
def obtener_color_dominante(image, bbox):
    x_min, y_min, x_max, y_max = bbox
    patch = image[y_min:y_max, x_min:x_max]
    if patch.size == 0:
        return (0, 0, 0)  # Devolver negro si el parche es inválido

    # Aplicar filtro de mediana
    patch = cv.medianBlur(patch, 5)
    hsv_patch = cv.cvtColor(patch, cv.COLOR_BGR2HSV)

    # Usar k-means para encontrar el color dominante
    pixels = hsv_patch.reshape(-1, 3)
    k = 3
    _, labels, centers = cv.kmeans(pixels.astype(np.float32), k, None,
                                (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0), 10, cv.KMEANS_RANDOM_CENTERS)
    dominant_color = centers[np.argmax(np.bincount(labels.flatten()))]
    return tuple(map(int, dominant_color))


def ajustar_brillo(image, factor=1.2):
    """Aumenta el brillo de la imagen."""
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * factor
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    bright_img = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bright_img


# Función para predecir el tipo de prenda
def predecir_prenda(image, bbox):
    x1, y1, x2, y2 = bbox
    roi = image[y1:y2, x1:x2]
    if roi.size == 0:
        return None
    roi = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    roi = cv.resize(roi, (28, 28))
    roi = roi / 255.0
    roi = roi.reshape(1, 28, 28, 1)
    prediction = model.predict(roi)
    return np.argmax(prediction)

<h2>Captura de video y procesamiento del algoritmo</h2>

In [20]:
# Inicialización de Mediapipe y OpenCV
cap = cv.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Ajustar el brillo de la imagen
        frame = ajustar_brillo(frame, factor=1.2)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False
        resultados = pose.process(image)
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        if resultados.pose_landmarks:
            puntos_de_referencia = resultados.pose_landmarks.landmark
            puntos_hito = [puntos_de_referencia[i] for i in puntos_interes]

            for idx, puntos_ref in zip(puntos_interes, puntos_hito):
                x = int(puntos_ref.x * image.shape[1])
                y = int(puntos_ref.y * image.shape[0])
                cv.circle(image, (x, y), 5, (245, 117, 66), -1)

            for start_idx, end_idx in conexiones_interes:
                if start_idx in puntos_interes and end_idx in puntos_interes:
                    start_point = (int(puntos_de_referencia[start_idx].x * image.shape[1]),
                                   int(puntos_de_referencia[start_idx].y * image.shape[0]))
                    end_point = (int(puntos_de_referencia[end_idx].x * image.shape[1]),
                                 int(puntos_de_referencia[end_idx].y * image.shape[0]))
                    cv.line(image, start_point, end_point, (245, 66, 230), 2)

            bbox_tronco_superior = dibujar_bounding_box(image, [puntos_de_referencia[i] for i in tronco_superior])
            bbox_tronco_inferior = dibujar_bounding_box_resize(image, [puntos_de_referencia[i] for i in tronco_inferior])
            bbox_pies = dibujar_bounding_box(image, [puntos_de_referencia[i] for i in pies])

            pred_tronco_superior = predecir_prenda(image, bbox_tronco_superior)
            pred_tronco_inferior = predecir_prenda(image, bbox_tronco_inferior)
            pred_pies = predecir_prenda(image, bbox_pies)
            if pred_tronco_superior or pred_tronco_inferior or pred_pies is not None:
                label_tronco_superior = index_to_label.get(pred_tronco_superior, "Camiseta")
                label_tronco_inferior = index_to_label.get(pred_tronco_superior, "Pantalon")
                label_pies = index_to_label.get(pred_tronco_superior, "Zapatilla")
                cv.putText(image, label_tronco_superior, (bbox_tronco_superior[0], bbox_tronco_superior[1] - 30), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv.putText(image, label_tronco_inferior, (bbox_tronco_inferior[0], bbox_tronco_inferior[1] - 30), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv.putText(image, label_pies, (bbox_pies[0], bbox_pies[1] - 30), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            
            color_intermedio_tronco_superior = obtener_color_dominante(image, bbox_tronco_superior)
            color_tronco_inferior = obtener_color_dominante(image, bbox_tronco_inferior)
            color_pies = obtener_color_dominante(image, bbox_pies)

            nombre_color_tronco_superior = obtener_nombre_color(color_intermedio_tronco_superior)
            nombre_color_tronco_inferior = obtener_nombre_color(color_tronco_inferior)
            nombre_color_pies = obtener_nombre_color(color_pies)

            cv.putText(image, nombre_color_tronco_superior, (bbox_tronco_superior[0], bbox_tronco_superior[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv.putText(image, nombre_color_tronco_inferior, (bbox_tronco_inferior[0], bbox_tronco_inferior[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv.putText(image, nombre_color_pies, (bbox_pies[0], bbox_pies[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv.imshow('Ventana de captura', image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━